# **Adversarial search strategies and Decision Trees**
## Inteligência Artificial
### Turma: PL
### Grupo: 
-
-
- Rodrigo Luz de Carvalho (202000184)

### Objetivo
O objetivo deste trabalho é implementar o algoritmo **Monte Carlo Tree Search (MCTS)** para jogar o jogo *Connect 4* utilizando o algoritmo *Upper Confidence Bound for Trees (UCT)* para a seleção do nó a ser expandido, e criar uma **Árvore de Decisões** através do procedimento *Iterative Dichotomiser 3 (ID3)*  para classificar 2 diferentes datasets. 

# Connect 4 e Monte Carlo Tree Search

**Connect Four** é um jogo de estratégia para dois jogadores. É jogado com 42 peças (21 de uma cor e 21 de outra) numa grade vertical com 7 colunas. Cada coluna pode conter no máximo 6 peças. Os dois jogadores jogam alternadamente colocando uma ficha numa das colunas. Quando uma ficha é colocada numa coluna, ela cai até atingir a parte inferior ou a ultima ficha nessa coluna. Um jogador vence quando pelo menos quatro de suas fichas estão alinhadas numa linha, coluna ou diagonal.

## Nossa representação
A interface do nosso jogo é representada no terminal, onde os dois jogadores são representados através de "X" - sempre o primeiro jogador - e "O" - sempre o segundo jogador -, sendo que "-" significa uma casa vazia. 

 `Turn: 13`<br>
 `0 1 2 3 4 5 6`<br>
 `- - - - - - -`<br>
 `- - - - - - -`<br>
 `- - - - - - -`<br>
 `O - - - - - -`<br>
 `X - O O - - -`<br>
 `X - X O - - -`<br>
 `X O X X O - X`<br>
 `Next to play: Monte Carlo as O`

Além disto, após o algoritmo do Monte Carlo Tree Search fazer sua jogada, é representado para cada uma das jogadas possíveis o número de vezes em que uma vitória foi alcançada a partir daqueles nós ("value"), o número de visitas nestes nós ("visits"), e a ratio entre o numero de vitórias e o numero de visitas - que é usada para determinar a jogada a seguir.

`Thinking...`<br>
`---> move: 2 value: 665 visits:775 ratio: 0.8580645161290322`<br>
`---> move: 5 value: 550 visits:690 ratio: 0.7971014492753623`<br>
`---> move: 6 value: 581 visits:713 ratio: 0.814866760168303`<br>
`---> move: 1 value: 515 visits:664 ratio: 0.7756024096385542`<br>
`---> move: 3 value: 637 visits:754 ratio: 0.8448275862068966`<br>
`---> move: 0 value: 503 visits:655 ratio: 0.76793893129771`<br>
`---> move: 4 value: 630 visits:749 ratio: 0.8411214953271028`<br>
`5000 iterations in 6 seconds`<br>
`Monte Carlo played in col: 2`

## Lógica do Jogo
A lógica do jogo é feita através da classe Game que define as funções utilizadas para a execução da partida. Estas funções são:
- player:
- drawboard:
- verifyCol:
- availableCollumns:
- playOneTurn:
- gameOver:


In [1]:
import numpy as np
NUM_ROW = 6
NUM_COL = 7
EMPTY = "-"
PLAYER_1_PIECE = "X"
PLAYER_2_PIECE = "O"

class game:
    def __init__(self):
        self.turn = 0 #contador do turno
        self.gameWinner = EMPTY #variavel que controla quem ganhou
        self.boardIsFull = False #retorna se a board esta cheia
        self.board = np.full([NUM_ROW, NUM_COL], EMPTY) #gera a board vazia
        self.tops = [0] * NUM_COL #array unidimensional que guarda quantas peças em cada coluna
        self.last_move = None #guarda o ultimo movimento p/ o MCTS

    """diz de quem é a vez"""
    def player(self):
        if self.turn % 2 == 0:
            return PLAYER_1_PIECE
        else:
            return PLAYER_2_PIECE

    """desenha a board"""
    def drawBoard(self):
        for i in range(7): print(i, end=" ") #imprime os numeros das colunas
        print() # coloca newline
        for line in self.board:
            for piece in line:
                print(piece, end=" ")
            print()# coloca newline
        return
    
    """verifica se uma coluna está disponível"""
    def verifyCol(self, col):
    #a coluna está disponível se o tamanho da pilha for menor que o total das linha
        if col >= NUM_COL or col < 0: # error handling quando a coluna não existe
            return False
        else:
            return self.tops[col] < NUM_ROW
        
    """lista de colunas disponíveis usada para o MTCS"""
    def availableCollumns(self):
        available = []
        for i, value in enumerate(self.board[0]):
            if value == EMPTY:
                available.append(i)
        return available
    
    """joga um turno"""
    def playOneTurn(self, col, piece): #joga um turno onde trata cada col como uma pilha
        if not self.verifyCol(col): #error handling ^2
            return False  # Col não está cheia
        
        # Coloca a coluna certa desde a base usando lógica da pilha
        row_to_place = NUM_ROW - 1 - self.tops[col]
        
        # Coloca a peça 
        self.board[row_to_place][col] = piece
        
        # Adiciona ao contador da pilha nessa coluna
        self.tops[col] += 1

        # atualiza o ultimo movimento (Usado no MCTS)
        self.last_move = col

        return True
    
    """verifica se um determinado estado é um estado final"""
    def gameOver(self):
        #retorna um boolean e altera o valor da variavel gameWinner
        for i in range(NUM_ROW):
            for j in range(NUM_COL):
                #verifica se a peça é X ou O
                piece = self.board[i, j] # type: ignore
                if piece == EMPTY:  
                    continue 
                #verifica em linha horizontal 
                if j <= NUM_COL - 4 and all(self.board[i, j + k] == piece for k in range(4)):
                    self.gameWinner = piece
                    return True
                #verifica em linha vertical 
                if i <= NUM_ROW - 4 and all(self.board[i + k, j] == piece for k in range(4)):
                    self.gameWinner = piece
                    return True
                #verifica a diagonal principal
                if i <= NUM_ROW - 4 and j <= NUM_COL - 4 and all(self.board[i + k, j + k] == piece for k in range(4)):
                    self.gameWinner = piece
                    return True
                #verifica a diagonal secundaria
                if i <= NUM_ROW - 4 and j >= 3 and all(self.board[i + k, j - k] == piece for k in range(4)):
                    self.gameWinner = piece
                    return True
        #verifica empate
        if all(not self.verifyCol(i) for i in range(NUM_COL)):  
            self.gameWinner = "Tie"
            return True

        return False

## Monte Carlo Tree Search (MCTS)
Na implementação do MCTS foi utilizado o algoritmo UCT (Upper Confidence Bound for Trees) para a seleção do nó a ser expandido. O valor de C no UCT, que balança exploração e exploitação, foi ajustado através dos testes que veremos mais a frente afim de otimizar os resultados.

Nesta implementação do MCTS adicionamos uma regra que avalia na expansão se há algum movimento em que resulte numa vitória imediata, e se sim, o MCTS fará esse movimento. Caso não haja,  ele verifica então se há condição de vitória imediata do adversário, e se sim, ele jogará nesta posição com o intuito de impedir a vitória.

In [4]:
import math 
import random
import copy
import time

"""O valor de C é baseado no resultado dos testes que realizaremos mais a diate
A limitação da quantidade de iterações está definida como um numero fixo, porém pode ser também definida por tempo"""

MCTS_TIME = 5.0 # em segundos
MAX_ITER = 10000
MCTS_C = 2.0

class Node:

    def __init__(self, state, parent=None):
        self.visits = 0
        self.value = 0
        self.state = state
        self.parent = parent
        self.children = []
        self.children_move = []

    def full_explored(self):
        return len(self.children) == len(self.state.availableCollumns())

    def update(self, v):
        self.value = self.value + v
        self.visits +=1
    
    def add_child(self, child, move):
        self.children.append(Node(child.state, parent=self))
        self.children_move.append(move)
    


class MCTS:
    def __init__(self, c=MCTS_C, max_iterations=MAX_ITER):
        self.processing_time = MCTS_TIME
        self.exploring_rate = c
        self.max_iterations = max_iterations

    """Função principal que retorna o movimento Final"""
    def getMove(self, state, statistics = True):
        self.aiPiece = state.player() # define a peça da AI

        new_state = copy.deepcopy(state) # cria copias do estado inicial
        node = Node(copy.deepcopy(new_state))

        start_time = time.process_time() # começa o timer do jogo
        iterations = 0
        run_time = 0
        
        # o loop continuar até o tempo acabar ou a quantidade máxima de iterações
        #while time.process_time() - start_time < self.processing_time:
        while iterations < self.max_iterations:
            nextNode = self.choose(node) # escolhe o próximo Node
            
            if (nextNode[1]): # se o node for uma vitória imediata, retorna esse node
                return node.children[-1].state.last_move # os movimentos feitos são salvos em last move, então é só ir ao last move da ultima child adicionada
            elif (nextNode[2]): # se o node do player for uma vitória imediata, retorna esse node, bloqueando o player
                return node.children[-1].state.last_move

            simValue = self.rollout(nextNode[0]) # faz a simulação da partida e retorna o resultado
            self.backpropagation(nextNode[0], simValue) # faz a backpropagation até a child escolhida
            iterations += 1
    
        run_time = int(time.process_time() - start_time)
        
        """Retorna a child com a maior razão entre vitórias e números de visitas"""
        melhorValor = float('-inf')
        for child in node.children: # itera sobre todas as children e escolhe a com melhor razão
            if child.visits == 0: # eliminar children que não foram visitadas, apesar de ser pouco provável
                continue
            elif child.value/child.visits > melhorValor:
                melhorValor = child.value/child.visits
                melhorChild = child
            if statistics:
                print(f"---> move: {child.state.last_move} value: {child.value} visits:{child.visits} ratio: {child.value/child.visits}") # estatísticas
        if statistics:
            print(f"{iterations} iterations in {run_time} seconds")
        
        return melhorChild.state.last_move # toda vez que um movimento é feito ele é salvo em last move, então é só ir ao lastmove da melhor child

    """Seleciona o próximo node para fazer rollout, retorna o node,True/False que diz se a proxima jogada é vitória imediata"""
    def choose(self, node):  
        if not node.full_explored(): # se todas as children do node não foram exploradas, retorna uma delas
            return self.expand(node)
        else: # se todas foram exploradas uma vez, retorna a melhor de acordo com UCBT
            node = self.bestChild(node) 
        return node
    
    """expande o nó inicial"""
    def expand(self, node):
        possible_actions = node.state.availableCollumns() # lista de ações possíveis
        notTaken = [] # lista de ações não tomadas

        for action in possible_actions: 
            if action not in node.children_move: # children_move lista as ações já tomadas
                notTaken.append(action)

        action = random.choice(notTaken) # escolhe aleatóriamente uma ação não tomada

        newchild = copy.deepcopy(node.state) # gera uma nova child baseado nas ações não tomadas
        newchild.playOneTurn(action,newchild.player()) # faz essa jogadaa
        newchild.turn += 1
        immediateAIWin = newchild.gameOver() # verifica se a AI vai ganhar nessa jogada

        testplayer = copy.deepcopy(node.state) # gera uma child teste que joga como o player e sera descartada
        testplayer.turn += 1 # incrementa o turno, jogando como o player
        testplayer.playOneTurn(action,testplayer.player()) # faz essa jogada como o player
        immediatePlayerWin = testplayer.gameOver()

        node.add_child(Node(newchild), action) # adiciona ao node inicial essa nova child
        return node.children[-1],immediateAIWin,immediatePlayerWin # retorna essa child para fazer o rollout e se ela é uma vitória imediata
    
    """Calcula a UCBT para cada child e retorna a melhor,
    o False serve para manter o output da função choose consistente e não altera nada em bestChild"""
    def bestChild(self, node):
        bestScore = float('-inf')
        bestChild = []

        for child in node.children:
            score = child.value/child.visits + self.exploring_rate * math.sqrt(2*math.log(node.visits)/child.visits) # UCBT
            if score == bestScore:
                bestChild.append(child)
            if score > bestScore:
                bestChild = [child]
                bestScore = score

        return random.choice(bestChild), False, False # escolhe aleatóriamente entre as que tem o maior valor
    
    """Realiza as simulações com jogadas aleatórias"""
    def rollout(self, node):
        new = node.state

        while not new.gameOver(): # enquanto o jogo não acabar continua o rollout
            new = copy.deepcopy(new)
            possible_actions = new.availableCollumns()

            if len(possible_actions) > 0:
                new.playOneTurn(random.choice(possible_actions), new.player())
                new.turn += 1

        if new.gameWinner == self.aiPiece: # se a ai vencer
            return 1
        elif new.gameWinner != self.aiPiece:
            return 0 # derrota da ai
        else:
            return -1 # empate

    """Faz a backpropagation"""
    def backpropagation(self, node, value):
        #node.state.turn +=1

        if value == -1: # empate
            while node != None:
                node.visits += 1
                node = node.parent

        elif value == 0: # derrota
            while node != None:
                node.value += value
                node.visits += 1
                #print(f"{node.value}; {node.state.player()} ")
                node = node.parent
                value = 1 - value

        else: # vitória
            while node != None:
                node.value += value
                node.visits += 1
                #print(f"{node.value}; {node.state.player()} ")
                node = node.parent

## Player Random
Para efeitos de comparação e controle, adicionamos um algoritmo que sempre faz jogadas completamente aleatórias dentro das jogadas possíveis.

In [ ]:
class ai_random: # Sempre faz um movimento Random
    def getMove(self, state):
        col = random.randrange(NUM_COL)
        while not state.verifyCol(col):
            col = random.randrange(NUM_COL)
        return col

## Player vs AI
Ao jogar contra a AI pode-se escolher ir primeiro ou segundo, e se jogará contra o algoritmo random ou o MCTS.

In [ ]:
NEW_GAME = True

def playerMove(piece): # movimento do jogador
    print(f"Next to play: Player as {piece}")
    try:  # error handling
        col = int(
            input("Choose in which col do you wanna play or press 9 to quit: ")
        )
    except:
        col = -1

    while not newGame.verifyCol(col):
        if col == 9:
            quit()  # opção para quitar
        try:  # error handling
            col = int(
                input(
                    "The col you selected is either full or invalid, choose another one or press 9 to quit: "
                )
            )
        except:
            col = -1
    newGame.playOneTurn(col,piece)


def aiMove(piece): # movimento da AI
    print(f"Next to play: {aiName} as {piece}")

    if aiName == "Monte Carlo":
        print("Thinking...")
        col = ai.getMove(newGame) # type: ignore

    else: 
        col = ai.getMove(newGame) # type: ignore
    
    print(f"{aiName} played in col: {col}")
    newGame.playOneTurn(col,piece)

def gameMode(): # seleciona o modo de jogo e inicializa a AI
    start = -1 # error handling
    while start not in range(3):
        try:  
            start = int(input("\nChoose the Game Mode\n0 => Player vs Player\n1 => AI Random\n2 => Monte Carlo\n: "))
        except:
            continue
    match start:
        case 0:
            ai = None
            aiName = None
        case 1:
            ai = ai_random()
            aiName = "Random"
        case 2:
            ai = MCTS()
            aiName = "Monte Carlo"
    return ai,aiName

def PlayOrder(): #decide quem vai primeiro
    try:  # error handling
        first = int(input("\nChoose 0 to go first or 1 to go second: "))
    except:
        first = -1
    if first not in range(2):
        PlayOrder()
    return first

def requestNewGame():
    bol = -1
    while bol < 0 or bol > 1:
        try:  # error handling
            bol = int(input("\nType 0 to quit or 1 to play again: "))
        except:
            bol = -1
    return (bol == 1) # retorna True se houver novo jogo


# GAME LOOP
while NEW_GAME:
    print("\nNew Game")
    newGame = game() # gera um novo objeto jogo
    
    start = gameMode() # seleciona o modo de jogo
    ai = start[0] # inicializa a AI
    aiName = start[1]

    if aiName != None:
        order = PlayOrder() # seleciona se começa ou não    
    else: 
        order = -1 # pvp não tem quem começa

    while newGame.gameWinner == EMPTY: #game loop
        print(f"\nTurn: {newGame.turn}")
        newGame.drawBoard()
        

        if order == -1: # controla de quem é a vez e faz o movimento
                playerMove(newGame.player())
        elif order == 0: # player joga primeiro
            if newGame.turn % 2 == 0:
                playerMove(newGame.player())
            else:
                aiMove(newGame.player())
        else: # player joga segundo
            if newGame.turn % 2 == 0:
                aiMove(newGame.player())
            else:
                playerMove(newGame.player())


        if newGame.gameOver(): # caso o jogo acabe
            if newGame.gameWinner == "Tie": # empate
                print("\n")
                newGame.drawBoard()
                print(f"\nIt's a Tie!")
            else: # vitória
                print("\n")
                newGame.drawBoard()
                print(f"\n{newGame.gameWinner} Won!")    
            NEW_GAME = requestNewGame() # define se haverá outro jogo

        else: # o jogo não acabou
            newGame.turn += 1 # próximo turno

quit()

## Definindo o melhor valor de C

Para definir o melhor valor da constante de exploitação / exploração foram feitos testes onde diferentes versões do MCTS jogaram umas contra as outras em sistemas de chave utilizando diferentes valores de C entre 0.2 e 2.  

Cada versão do MCTS jogou 2 jogos contra a adversária, alternando quem começou (dado que há uma vantagem para quem começa):

`0.2 - 0.4 | 0.6 - 0.8 | 1.0 - 1.2 | 1.4 - 1.6 | 1.8 - 2.0`

As versões com o maior número de vitórias jogaram entre si até haver uma única vitóriosa, que em seguida jogou contra duas versões com +0.1 e -0.1 afim de refinar o resultado.

In [ ]:
"""Gameloop de uma AI contra outra AI"""
def gameLoop(c1,c2):
    print(f"\nNovo Jogo entre {c1} e {c2}")
    newGame = game() # gera um novo objeto jogo
    
    ai1 = MCTS(c1,10) #100.000 iterações por jogada
    aiName1 = "MCTS: c = " + str(c1)
    ai2 = MCTS(c2,10) #100.000 iterações por jogada
    aiName2 = "MCTS: c = " + str(c2)

    while newGame.gameWinner == EMPTY: #game loop
        
        if newGame.turn % 2 == 0:
            col = ai1.getMove(newGame,False) # type: ignore
            newGame.playOneTurn(col,newGame.player())
        else:
            col = ai2.getMove(newGame,False) # type: ignore
            newGame.playOneTurn(col,newGame.player())

        if newGame.gameOver(): # caso o jogo acabe
            if newGame.gameWinner == "Tie": # empate
                print("\n")
                newGame.drawBoard()
                print(f"\nIt's a Tie!")
                return 0,0
            elif newGame.gameWinner == "X":
                print("\n")
                newGame.drawBoard()
                print(f"\nX Won")
                return c1,c2
            else:
                print("\n")
                newGame.drawBoard()
                print(f"\nO Won")
                return c2,c1
            
        else: # o jogo não acabou
            newGame.turn += 1 # próximo turno
    
    return 0,0 #failsafe

"""realiza 2 jogos entre MCTSs com valores diferentes de c (c1 e c2) e retorna quem perdeu
caso resulte num empate, retorna o menor valor como perdedor"""
def worstC(c1,c2):
    result1 = gameLoop(c1,c2)
    result2 = gameLoop(c2,c1)

    if result1[0] == 0 and result2[0] != 0: #(0,vitoria)
        return result2[1]
    elif result2[0] == 0 and result1[0] != 0: #(vitoria,0)
        return result1[1]
    elif result1[0] == 0: #(0,0)
        return c1
    elif result1[0] == result2[0]: #(vitoria,vitoria)
        return result1[1]
    else: #(vitoria cx, vitoria cy)
        return c1

"""executa diversos jogos com os valores de C entre 0.2 e 2.0 e retorna o melhor valor"""
bestC = []
for i in range (2,21,2):
    bestC.append(i/10)

# primeiro round da chave
for i in range (2,20,4):
    c1 = i/10
    c2 = (i+2)/10
    bestC.remove(worstC(c1,c2))

bestC.remove(worstC(bestC[0],bestC[1])) # segundo round
bestC.remove(worstC(bestC[2], bestC[3]))
bestC.remove(worstC(bestC[0],bestC[2])) # terceiro round
bestC.remove(worstC(bestC[0],bestC[1])) # último round

bestC.append(((bestC[0]*10)-1)/10) # evitar lidar com tipo Decimal
bestC.append(((bestC[0]*10)+1)/10)

bestC.remove(worstC(bestC[1],bestC[2])) # refinamento dos resultados
bestC.remove(worstC(bestC[0],bestC[1])) # resultado final

print(f"O melhor valor de C é: {bestC[0]}")    


Novo Jogo entre 0.2 e 0.4


0 1 2 3 4 5 6 
- - - - - - - 
- - - - - - - 
- - - - - - - 
- - - - - - - 
- O - - - - - 
X X X X O - O 

X Won

Novo Jogo entre 0.4 e 0.2


0 1 2 3 4 5 6 
- - - O - - - 
- - - X - - - 
- X - X - O X 
X O X X - O X 
O X O O - X O 
X O O X O X O 

X Won

Novo Jogo entre 0.6 e 0.8


0 1 2 3 4 5 6 
- - - - - - - 
- - - - - - - 
X - O X - - - 
O O X O X - - 
O X X X O - - 
O X X X O O O 

O Won

Novo Jogo entre 0.8 e 0.6


0 1 2 3 4 5 6 
- X - O X - - 
X X - X O - - 
O O - X X - X 
O X - O O O O 
X X - X O X O 
O X - X O O O 

O Won

Novo Jogo entre 1.0 e 1.2


0 1 2 3 4 5 6 
- - - - - - - 
- - - - - - - 
O - - - - - - 
X O X O - - - 
X O O X O - - 
X X X O O - X 

O Won

Novo Jogo entre 1.2 e 1.0


0 1 2 3 4 5 6 
O - - - - - - 
O - - - O X - 
O O - O X X - 
X X X X O O - 
X X O X O O - 
X O X X X O - 

X Won

Novo Jogo entre 1.4 e 1.6


0 1 2 3 4 5 6 
- - - - - X - 
- - - O - O - 
X - X O O O - 
O O X X X O - 
X X X O O X O 
X O O X X O X 

O Won

Novo Jogo en